In [1]:
!pip install neo4j langchain langchain-community

  Using cached langchain-0.3.27-py3-none-any.whl.metadata (7.8 kB)
  Using cached jsonpatch-1.33-py2.py3-none-any.whl.metadata (3.0 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached typing_inspection-0.4.1-py3-none-any.whl.metadata (2.6 kB)
  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached httpx_sse-0.4.1-py3-none-any.whl.metadata (9.4 kB)
  Using cached aiohappyeyeballs-2.6.1-py3-none-any.whl.metadata (5.9 kB)
  Using cached aiosignal-1.4.0-py3-none-any.whl.metadata (3.7 kB)
  Using cached marshmallow-3.26.1-py3-none-any.whl.metadata (7.3 kB)
  Using cached typing_inspect-0.9.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached mypy_extensions-1.1.0-py3-none-any.whl.metadata (1.1 kB)
Using cached langchain-0.3.27-py3-none-any.whl (1.0 MB)
Using cached jsonpatch-1.33-py2.py3-none-any.whl (12 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8

In [2]:
!pip install --upgrade pip


  Using cached pip-25.2-py3-none-any.whl.metadata (4.7 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 21.4 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 25.1.1
    Uninstalling pip-25.1.1:
      Successfully uninstalled pip-25.1.1


In [3]:
from neo4j import GraphDatabase

# Neo4j Desktop에서 설정한 데이터베이스의 접속 정보를 입력합니다.
# 기본값은 bolt://localhost:7687이며, 사용자 이름과 비밀번호는 설정한 것으로 입력합니다.
uri = "bolt://localhost:7687" # <<!nav>>볼트<<!/nav>> 프로토콜을 사용하므로 "bolt://"로 시작합니다 [2, 4].
user = "neo4j" # 또는 사용자가 설정한 사용자 이름
password = "ekvmsp2384" # 사용자가 설정한 비밀번호

# Driver 객체를 생성합니다.
driver = GraphDatabase.driver(uri, auth=(user, password))

# 세션을 생성하여 쿼리를 실행합니다.
with driver.session() as session:
    # 간단한 노드 생성 쿼리 예시
    result = session.execute_write(lambda tx: tx.run("MERGE (a:Person {name: $name}) RETURN a", name="Alice").data())

    print("쿼리 결과:", result)

# Driver를 닫습니다 (애플리케이션 종료 시).
driver.close()

쿼리 결과: [{'a': {'name': 'Alice'}}]


In [15]:
q_load_articles_local = """
LOAD CSV WITH HEADERS
FROM 'file:///20251001_Neo4j_Sample.txt'  // 로컬 Neo4j의 'import' 폴더에 파일을 넣어야 합니다.
AS 행 
FIELDTERMINATOR ';'
MERGE (a:블로그포스트 {제목:행.제목})
SET a.요약 = 행.요약,
    a.발행일 = date(행.발행일)
FOREACH (작성자 in split(행.작성자, ',') | 
    MERGE (p:개발자 {이름:trim(작성자)})
    MERGE (p)-[:작성하다]->(a))
FOREACH (주제 in [행.주제] | 
    MERGE (t:주제 {이름:trim(주제)})
    MERGE (a)-[:주제이다]->(t))
FOREACH (세부주제 in [행.세부주제] | 
    MERGE (st:세부주제 {이름:trim(세부주제)})
    MERGE (a)-[:세부주제이다]->(st))
"""


In [16]:

# 4. 세션 생성 및 쿼리 실행 (execute_write 사용)
def load_csv_data(tx):
    # 쿼리 실행
    result = tx.run(q_load_articles_local)
    
    # !!! 수정된 부분: .consume()을 사용하여 결과를 소모하고 요약 정보를 가져옵니다.
    summary = result.consume()
    
    # 요약 정보에서 counters (카운터)를 추출합니다.
    return f"쿼리 실행 완료. 통계: {summary.counters}"

# Driver 객체를 생성합니다.
driver = GraphDatabase.driver(uri, auth=(user, password))

with driver.session() as session:
    print("⏳ 데이터 로딩 시작...")
    # execute_write를 사용하여 트랜잭션 내에서 쿼리 실행
    summary = session.execute_write(load_csv_data)
    print(f"🚀 데이터 로딩 성공! {summary}")

# 5. Driver 닫기
driver.close()

⏳ 데이터 로딩 시작...
🚀 데이터 로딩 성공! 쿼리 실행 완료. 통계: SummaryCounters{labels_added: 56, relationships_created: 82, nodes_created: 56, properties_set: 94, contains_updates: True, contains_system_updates: False}


In [17]:
!pip -q install langchain-aws


In [19]:
from langchain_community.graphs import Neo4jGraph

graph = Neo4jGraph(
    url=uri, 
    username=user,
    password=password,
)

In [ ]:
from langchain.chains import GraphCypherQAChain
from langchain_aws import ChatBedrock

graph.refresh_schema()

cypher_chain = GraphCypherQAChain.from_llm(
    cypher_llm = ChatBedrock(model_id="anthropic.claude-3-5-sonnet-20240620-v1:0"),
    qa_llm = ChatBedrock(model_id="anthropic.claude-3-5-sonnet-20240620-v1:0"), 
    graph=graph,
    verbose=True,
    allow_dangerous_requests=True
)

In [20]:
from langchain_community.graphs import Neo4jGraph

graph = Neo4jGraph(
    url=uri, 
    username=user,
    password=password,
)
graph.refresh_schema()
print(graph.get_schema)



Node properties:
블로그포스트 {제목: STRING, 요약: STRING, 발행일: DATE}
개발자 {이름: STRING}
주제 {이름: STRING}
세부주제 {이름: STRING}
Relationship properties:

The relationships:
(:블로그포스트)-[:주제이다]->(:주제)
(:블로그포스트)-[:세부주제이다]->(:세부주제)
(:개발자)-[:작성하다]->(:블로그포스트)


In [23]:
!pip install langchain_openai

  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 948.6/948.6 kB 21.4 MB/s  0:00:00
Using cached distro-1.9.0-py3-none-any.whl (20 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 997.1/997.1 kB 7.6 MB/s  0:00:00
Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7/7 [langchain_openai]━ 5/7 [openai]


In [ ]:
import os
# OpenAI 관련 클래스를 임포트합니다.
from langchain_openai import ChatOpenAI
from langchain.chains import GraphCypherQAChain
# 이미 정의된 graph 객체를 사용합니다. (Neo4jGraph 인스턴스)

# (필수) OPENAI_API_KEY 환경 변수가 설정되어 있어야 합니다.
os.environ["OPENAI_API_KEY"] = ""
# Neo4j 스키마를 새로 고칩니다. (APOC 오류 해결 후 정상 작동한다고 가정)
graph.refresh_schema()

# OpenAI 모델 인스턴스 생성
# Cypher 생성 및 최종 답변 생성에 모두 Claude 3.5 Sonnet과 비슷한 수준의 모델을 사용합니다.
# gpt-4o 또는 gpt-4-turbo가 복잡한 Cypher 생성에 적합합니다.
openai_llm = ChatOpenAI(
    model="gpt-4.1",  # 최신 고성능 모델 사용 권장
    temperature=0  # Cypher 생성 시 창의성보다 정확성이 중요하므로 0을 사용합니다.
)

# GraphCypherQAChain 구성
cypher_chain = GraphCypherQAChain.from_llm(
    cypher_llm=openai_llm,  # Cypher 쿼리 생성에 OpenAI 모델 사용
    qa_llm=openai_llm,      # 최종 답변 생성에 OpenAI 모델 사용
    graph=graph,
    verbose=True,
    allow_dangerous_requests=True,
)

print("✅ GraphCypherQAChain이 OpenAI 모델(gpt-4o)로 성공적으로 구성되었습니다.")
# 이제 cypher_chain.invoke({"query": "질문 내용"}) 형태로 사용할 수 있습니다.

✅ GraphCypherQAChain이 OpenAI 모델(gpt-4o)로 성공적으로 구성되었습니다.


In [27]:
cypher_chain.invoke(
    {"query": "이지훈이 발표한 블로그는 총 몇 편인가요?"}
)



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (d:개발자 {이름: "이지훈"})-[:작성하다]->(b:블로그포스트)
RETURN count(b) AS 블로그포스트_수
Full Context:
[{'블로그포스트_수': 7}]

> Finished chain.


{'query': '이지훈이 발표한 블로그는 총 몇 편인가요?', 'result': '이지훈이 발표한 블로그는 총 7편입니다.'}

In [28]:
cypher_chain.invoke(
    {"query": "관심사가 같은 사람끼리 커피챗을 짝지어주려 합니다. 모든 사람이 포함되도록 그룹을 짝지어주세요."}
)



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (d1:개발자)-[:작성하다]->(:블로그포스트)-[:주제이다]->(t:주제)<-[:주제이다]-(:블로그포스트)<-[:작성하다]-(d2:개발자)
WHERE d1 <> d2
WITH DISTINCT d1, d2, t
RETURN d1.이름 AS 개발자1, d2.이름 AS 개발자2, t.이름 AS 공통주제
ORDER BY 개발자1, 개발자2, 공통주제
Full Context:
[{'개발자1': '김민지', '개발자2': '김철수', '공통주제': '모바일 개발'}, {'개발자1': '김민지', '개발자2': '김철수', '공통주제': '백엔드 개발'}, {'개발자1': '김민지', '개발자2': '김철수', '공통주제': '프론트엔드 개발'}, {'개발자1': '김민지', '개발자2': '박영희', '공통주제': '백엔드 개발'}, {'개발자1': '김민지', '개발자2': '박영희', '공통주제': '프론트엔드 개발'}, {'개발자1': '김민지', '개발자2': '이상훈', '공통주제': '프론트엔드 개발'}, {'개발자1': '김민지', '개발자2': '이지훈', '공통주제': '백엔드 개발'}, {'개발자1': '김민지', '개발자2': '정미경', '공통주제': '백엔드 개발'}, {'개발자1': '김민지', '개발자2': '최유진', '공통주제': '모바일 개발'}, {'개발자1': '김민지', '개발자2': '최유진', '공통주제': '백엔드 개발'}]

> Finished chain.


{'query': '관심사가 같은 사람끼리 커피챗을 짝지어주려 합니다. 모든 사람이 포함되도록 그룹을 짝지어주세요.',
 'result': '김민지-김철수(모바일 개발, 백엔드 개발, 프론트엔드 개발), 김민지-박영희(백엔드 개발, 프론트엔드 개발), 김민지-이상훈(프론트엔드 개발), 김민지-이지훈(백엔드 개발), 김민지-정미경(백엔드 개발), 김민지-최유진(모바일 개발, 백엔드 개발)로 그룹을 짝지을 수 있습니다.'}